In [ ]:
import requests
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

with open('header.json') as fi:
    credentials = json.load(fi)

header = credentials['header']

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 

parameters = {
    'start':'1',
    'limit':'5000',
    'convert':'USD'
}

session = Session()
session.header.update(header)

try:
    response = session.get(url, params=parameters)
    data = json.loads(response.text)
    print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)